# Image region of interest selection
This code helps you select multiple region of interests (ROIs) and then automatically saves them as .npy files in the same directory.

## Functions
1. Reads all the .npy files available in the directory in the numerical order of file name.
2. Opens .npy file in a loop and allows the user to select multiple ROIs.
3. Coordinates of each ROI is saved in a .txt file in the same directory.
4. Once ROIs are selected in all the images. .txt files are read by the code automatically.
5. Then ROIs are cropped and saved in the same directory.

## How to use
1. Make sure all the necessary python libraries are installed.
2. Copy the path to the image directory and past in directory_path.
3. Run the cell and a new dialog box should appear with the image.
4. Start selecting the ROIs in the images. You can select multiple ROIs by clicking and dragging the cursor. Take care not to randomly click on the image
otherwise point corrdinates are recorded and unnecessary files are created. To extract point spectra refer to point_roi.ipynb file in the dataset.
5. This code will run until all the images in the directory have been processed for ROI selection.
6. Once this script has finished. You should see .txt files in the directory and corresponding .npy ROIs files.

## Keyboard shortcuts
1. r: If you want to reset the selected pROIs in an image press 'r' in your keyboard. The previous ROIs are deleted and now you can select new ROIs.
2. c: Once you have marked all the ROIs in an image press 'c' to continue to the next image.
3. ESC: stops the processing in between. 

# Import necessary libraries

In [1]:
# importing the required libraries
import glob
import re
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.color import rgb2gray

# Set the directory path here

In [2]:
# Specify the directory path to the image folder
directory_path = '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/'

# ROI selection, cropping and saving happens here

In [3]:
# Create a list of .npy files in the directory
npy_files = glob.glob(directory_path + '/*.npy')

# Function to extract numbers from the file name
def extract_number(f):
    s = re.findall("\d+",f)
    return (int(s[0]) if s else -1,f)

# Sort the list of .npy files in numerical order
npy_files = sorted(npy_files, key=extract_number)

# Print the sorted list of .npy files
print(npy_files)


def click_and_crop(event, x, y, flags, param):
    # grab references to the global variables
    global rectangles, current_rectangle, cropping, image

    # if the left mouse button was clicked, record the starting
    # (x, y) coordinates and indicate that cropping is being
    # performed
    if event == cv2.EVENT_LBUTTONDOWN:
        current_rectangle = [(x, y)]
        cropping = True

    # draw the rectangle while the mouse is moving and the left mouse button is down
    elif event == cv2.EVENT_MOUSEMOVE and cropping:
        image_temp = image.copy()
        cv2.rectangle(image_temp, current_rectangle[0], (x, y), (0, 255, 0), 2)
        cv2.imshow("image", image_temp)
        cv2.waitKey(1)  # Add a small delay to allow the window to refresh

    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates
        current_rectangle.append((x, y))
        cropping = False # cropping is finished

        # draw a rectangle around the region of interest
        cv2.rectangle(image, current_rectangle[0], current_rectangle[1], (0, 255, 0), 2)
        cv2.imshow("image", image)
        rectangles.append(tuple(current_rectangle))

exit_loop = False

for file_path in npy_files:
    
    if exit_loop:
        break
    # load the image, clone it, and setup the mouse callback function
    rectangles = []
    current_rectangle = []
    cropping = False

    image = np.load(file_path)
    image = rgb2gray(image[:,:,[50,80,150]])
    image = ((image - image.min()) * (255 / (image.max() - image.min()))).astype(np.uint8)


    # Convert color from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    clone = image.copy()
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", click_and_crop)

    # keep looping until the 'q' key is pressed
    while True:
        # display the image and wait for a keypress
        cv2.imshow("image", image)
        key = cv2.waitKey(1) & 0xFF

        # if the 'r' key is pressed, reset the cropping region
        if key == ord("r"):
            image = clone.copy()
            rectangles = []
            current_rectangle = []
            cropping = False

        # if the 'c' key is pressed, break from the loop
        elif key == ord("c"):
            break
        
                # press esc to stop the program
        elif key == 27:
            exit_loop = True
            break

    # close all open windows
    cv2.destroyAllWindows()


    # write rectangles to file
    # use os.path to get the base name of the file and replace the extension with .txt
    output_file_path = os.path.splitext(file_path)[0] + '.txt'
    with open(output_file_path, 'w') as f:
        for rectangle in rectangles:
            f.write(str(rectangle) + '\n')
            
            
            
# Create a list of .npy files in the directory
txt_files = glob.glob(directory_path + '/*.txt')

# Function to extract numbers from the file name
def extract_number(f):
    s = re.findall("\d+",f)
    return (int(s[0]) if s else -1,f)

# Sort the list of .npy files in numerical order
txt_files = sorted(txt_files, key=extract_number)

# Print the sorted list of .npy files
print(txt_files)


# For each .npy and .txt file pair
for npy_file_name, txt_file_name in zip(npy_files, txt_files):
    # Load the .npy file
    npy_file = np.load(npy_file_name)

    # Open the corresponding .txt file
    with open(txt_file_name, 'r') as f:
        lines = f.readlines()

    # For each line in the .txt file
    for i, line in enumerate(lines):
        # Parse the coordinates
        coords = eval(line.strip())
        x1, y1 = coords[0]
        x2, y2 = coords[1]

        # Crop the ROI from the .npy file
        roi = npy_file[y1:y2, x1:x2]

        # Save the ROI as a separate .npy file
        base_name = os.path.splitext(npy_file_name)[0]
        np.save(f'{base_name}_{i+1}.npy', roi)

['/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_1.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_2.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_3.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_4.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_5.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_6.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_7.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_8.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_9.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_10.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_11.npy', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_12.npy', '/Users/bill

2024-02-27 20:21:15.139 Python[7055:372583] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


['/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_1.txt', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_2.txt', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_3.txt', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_4.txt', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_5.txt', '/Users/billygrahamram/Documents/OpenSourceData/Weeds/waterhemp/waterhemp_6.txt']


# ROI validation
Once the above script has completed you can check the individual ROI by running the following code. You will need to save replace the roi file path in the below code.

In [ ]:
roi_directory = '/Crops/sugarbeet/sugarbeet_20_4.npy'
file = np.load(roi_directory)
file = file[:,:,[80]]
plt.imshow(file, cmap='gray')
plt.show()